In [1]:
import tensorflow as tf
from keras.models import load_model
import cv2
import numpy as np

In [2]:
model=tf.keras.models.load_model('calculator_model.h5')

In [3]:
def testing(img):
    img=cv2.bitwise_not(img)
    img=cv2.resize(img,(28,28))
    img=img.reshape(1,28,28,1)
    img=img.astype('float32')
    img=img/255.0

    return model.predict(img)

def img_change():
    labimg = Image.open('Contours.png')
    labimg = ctk.CTkImage(dark_image = labimg, size = (width//5,height//5))

def num_to_sym(x):
    if x == 10:
        return '+'
    if x == 11:
        return '-'
    if x == 12:
        return '*'
    if x == 13:
        return '/'
    if x == 14:
        return '('
    if x == 15:
        return ')'
    if x == 16:
        return '.'
    else:
        return str(x)

def solve_exp(preds):
    ans = ""
    for x, y, w, h, ind, acc in preds:
        ans += ind
        print(ind + " " + str(acc))
        
    try:
        fin = eval(ans)
        fin = float(f"{fin:.4f}")
    
        txt.delete('1.0', ctk.END) # Delete prev expression
        sol.delete('1.0', ctk.END) # Delete prev calculations
    
        txt.insert(ctk.INSERT, "{}".format(ans))
        sol.insert(ctk.INSERT, "= {}".format(fin))
        
    except Exception:
        txt.delete('1.0', ctk.END) # Delete prev expression
        sol.delete('1.0', ctk.END) # Delete prev calculations
    
        txt.insert(ctk.INSERT, "{}".format(ans))
        sol.insert(ctk.INSERT, "Invalid expression")

In [4]:
import os

directory = os.getcwd()
imsave = directory+"\\Images\\"

print("Images used by CNN to predict individual numbers are stored here: " + imsave)

Images used by CNN to predict individual numbers are stored here: D:\ML\Calculator With CNN\Images\


In [5]:
equation = []


def add_padding(img, pad=5):
    h, w = img.shape[:2]
    padded_img = np.ones((h + pad * 2, w + pad * 2, 3), dtype=np.uint8) * 255
    padded_img[pad:pad + h, pad:pad + w] = img
    return padded_img


def preprocess_image(img):
    img = cv2.GaussianBlur(img, (5, 5), 5)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, bw = cv2.threshold(gray_img, 200, 255, cv2.THRESH_BINARY)
    return gray_img, bw


def find_and_sort_contours(bw):
    bw = cv2.bitwise_not(bw)
    cnts, _ = cv2.findContours(bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])
    return cnts


def process_contours(cnts, gray_img, testing, num_to_sym):
    preds = []
    for cnt in cnts:
        x, y, w, h = cv2.boundingRect(cnt)
        cropped_img = gray_img[y:y + h, x:x + w]

        if abs(h) > 1.25 * abs(w):
            pad = 3 * (h // w) ** 3
            cropped_img = cv2.copyMakeBorder(cropped_img, 0, 0, pad, pad, cv2.BORDER_CONSTANT, value=255)

        if abs(w) > 1.1 * abs(h):
            pad = 3 * (w // h) ** 3
            cropped_img = cv2.copyMakeBorder(cropped_img, pad, pad, 0, 0, cv2.BORDER_CONSTANT, value=255)

        resized_img = cv2.resize(cropped_img, (28, 28))
        padded_img = cv2.copyMakeBorder(resized_img, 2, 2, 2, 2, cv2.BORDER_CONSTANT, value=255)
        cv2.imwrite(f'{imsave}\\img_{x}_{y}.png', padded_img)

        predi = testing(padded_img)
        ind = np.argmax(predi[0])
        acc = float(f"{predi[0][ind] * 100:.2f}")

        preds.append((x, y, w, h, num_to_sym(ind), acc))
    return preds


def draw_predictions(img, preds, num_to_sym):
    for x, y, w, h, pred, acc in preds:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        text_y = y - 10 if y > 20 else y + h + 20
        cv2.putText(img, f"{num_to_sym(pred)}", (x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 2.5, (255, 0, 0), 3)
        cv2.putText(img, f"{acc}%", (x, text_y + 40), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)
    cv2.imwrite('Contours.png', img)
    img_change()


def preds_observer():
    preds = []
    
    image1.save("image1.png")
    img_1 = cv2.imread("image1.png")
    image2.save("image2.png")
    img_2 = cv2.imread("image2.png")

    img_1 = add_padding(img_1)
    gray_img_1, bw_1 = preprocess_image(img_1)
    cnts_1 = find_and_sort_contours(bw_1)

    img_2 = add_padding(img_2)
    gray_img_2, bw_2 = preprocess_image(img_2)
    cnts_2 = find_and_sort_contours(bw_2)

    global equation
    if len(cnts_2) >= 1:
        preds_1 = process_contours(cnts_1, gray_img_1, testing, num_to_sym)
        preds_2 = process_contours(cnts_2, gray_img_2, testing, num_to_sym)
        preds = preds_1 + preds_2
        equation = equation + preds
    else:
        preds_1 = process_contours(cnts_1, gray_img_1, testing, num_to_sym)
        equation = equation + preds_1

    # To show equation
    eq = ""
    for x, y, w, h, ind, acc in preds:
        eq += ind
    txt.delete('1.0', ctk.END)
    txt.insert(ctk.INSERT, "{}".format(eq))
    draw_predictions(img_1, preds_1, num_to_sym)
    clear_slide()


def mod():  
    preds_observer()
    return equation


def calculate():
    preds = mod()
    solve_exp(preds)
    global equation
    equation.clear()
    

def paint(event):
    d = 15
    x1, y1 = (event.x - d), (event.y - d)
    x2, y2 = (event.x + d), (event.y + d)
    canv.create_oval(x1, y1, x2, y2, fill="black", width=25)
    draw.line([x1, y1, x2, y2], fill="black", width=35)


def paint_sym(event):
    d = 15
    x1, y1 = (event.x - d), (event.y - d)
    x2, y2 = (event.x + d), (event.y + d)
    canv_1.create_oval(x1, y1, x2, y2, fill="black", width=10)
    draw_1.line([x1, y1, x2, y2], fill="black", width=20)


def clear_slide():
    canv.delete('all')
    draw.rectangle((0, 0, width, height), fill=(255, 255, 255, 0))
    canv_1.delete('all')
    draw_1.rectangle((0, 0, width, height), fill=(255, 255, 255, 0))


def clear():
    clear_slide()
    txt.delete('1.0', ctk.END)
    sol.delete('1.0', ctk.END)

In [6]:
red = (0, 0, 225)
green = (0, 230, 0)
blue = (225, 0, 0)

In [ ]:
from PIL import ImageTk, Image, ImageDraw
import PIL
import customtkinter as ctk

width = 1500
height = 600

root = ctk.CTk()
root.title('HANDWRITING CALCULATOR')

# Canvas for drawing numbers
canv = ctk.CTkCanvas(root, width=width, height=height, bg='white', highlightthickness=10, highlightbackground="black", highlightcolor="black")
canv.grid(row=0, column=0, columnspan=6, padx=50, pady=50)
canv.bind("<B1-Motion>", paint)
canv_1 = ctk.CTkCanvas(root, width=width//8, height=height//3, bg="white", highlightthickness=8, highlightbackground="black", highlightcolor="black")
canv_1.grid(row=0, column=6, rowspan=2)
canv_1.bind("<B1-Motion>", paint_sym)

white = (255, 255, 255)
image1 = PIL.Image.new("RGB", (width, height), white)
draw = ImageDraw.Draw(image1)
image2 = PIL.Image.new("RGB", (width, height), white)
draw_1 = ImageDraw.Draw(image2)

your_font = "Bahnschrift"

# Text boxes
text_font = ctk.CTkFont(family=your_font, size=27)
txt = ctk.CTkTextbox(root, exportselection=0,
              padx=10, pady=10, height=height//10, width=width//5, font=text_font)
txt.grid(row=2, column=1, padx=0, pady=3)

text_font = ctk.CTkFont(family=your_font, size=30, weight='bold')
sol = ctk.CTkTextbox(root, exportselection=0,
              padx=10, pady=10, height=height//10, width=width//5, font=text_font, text_color='#3085ff')
sol.grid(row=2, column=3, padx=0, pady=3)

equal_img = Image.open("./Assets/equals-symbol.png")
clear_img = Image.open("./Assets/cancel.png")
add_img = Image.open("./Assets/slide-addition.png")

# Buttons
button_font = ctk.CTkFont(family=your_font, size=25)
Pred = ctk.CTkButton(root, image=ctk.CTkImage(equal_img), text="", text_color="black", command=calculate, fg_color = 'white', hover_color='#007dfe',font = button_font,
                    height = height//10, corner_radius=32)
Clr = ctk.CTkButton(root, image=ctk.CTkImage(clear_img), text="", text_color="black", command=clear, fg_color = 'white', hover_color='#dd0000', font = button_font,
                    height = height//10, corner_radius=32)
Add_slide = ctk.CTkButton(root, image=ctk.CTkImage(add_img), text="", text_color="black", command=preds_observer, fg_color = 'white', hover_color='#39e75f', font = button_font,
                    height = height//10, corner_radius=32)

Pred.grid(row=2, column=2, padx=50, pady=5, sticky='ew')
Add_slide.grid(row=3, column=2, padx=50, pady=5, sticky='ew')
Clr.grid(row=4, column=2, padx=50, pady=5, sticky='ew')

root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 597ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
2 69.12
4 99.73
+ 99.99
4 99.36
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2 99.73
4 95.67
+ 100.0
3 99.99
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2 99.33
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2 99.37
4 99.65
* 100.0
8 100.0
